<a href="https://colab.research.google.com/github/Bombbom/ML4SecurityProject/blob/main/BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BiLSTM